In [1]:
import pandas as pd

df = pd.read_csv("spa-eng/spa.txt", sep="\t", header=None, names=["English", "Spanish", "Attribution"], encoding="utf-8")

print(df.head())


  English  Spanish                                        Attribution
0     Go.      Ve.  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1     Go.    Vete.  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2     Go.    Vaya.  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3     Go.  Váyase.  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4     Hi.    Hola.  CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [2]:
df.shape

(139013, 3)

In [3]:
l_eng = []
for i in df['English']:
    l_eng.append(len(i.split(" ")))
max(l_eng)

70

In [4]:
l_spa = []
for i in df['Spanish']:
    l_spa.append(len(i.split(" ")))
max(l_spa)

68